# Pydantic

Pydantic is a lib that simplifies data validation and settings management by using type annotation. It should be used for the following activities:

* Validate Input Data
* Perform Type Conversion
* Provide Easy Serialization

When using default type checking in Python it is static, and will not be validated at runtime. However, if you use Pydantic with type checking validation of types will happen at runtime, or conversion will happen.

In [9]:
from pydantic import BaseModel  

# here pydantic will automatically generate the init function for you. 
class User(BaseModel):
    id: int
    name: str
    email: str
    age: int = None

user_data = {"id": "123", "name":"Alice", "email": "alice@exmp.com", "age":"30"}
user = User(**user_data)
user

User(id=123, name='Alice', email='alice@exmp.com', age=30)

Pydantic will automatically convert data to the correct data type. For instance id and age are passed as string, but lets check what the datatype.

In [7]:
isinstance(user.id, int), isinstance(user.id, int)

(True, True)

By default conversion or coersion happens at runtime, but you can use strict types such as those used below, and static type checking is now dynamic type checking. 

In [18]:
from pydantic import BaseModel, StrictInt, StrictStr, ValidationError

class StrictExample(BaseModel):
    some_int: StrictInt
    some_str: StrictStr

se = StrictExample(some_int="1", some_str=2)

ValidationError: 2 validation errors for StrictExample
some_int
  value is not a valid integer (type=type_error.integer)
some_str
  str type expected (type=type_error.str)

## Serialization

You can also use Pydantic for serialization. For instance you can easily create dictionaries, or json objects like so:

In [11]:
user.dict()

{'id': 123, 'name': 'Alice', 'email': 'alice@exmp.com', 'age': 30}

In [10]:
user.json()

'{"id": 123, "name": "Alice", "email": "alice@exmp.com", "age": 30}'

## Advanced Types

You can get a bit more advanced when creating fields:

In [12]:
from pydantic import BaseModel, Field, ValidationError

class Product(BaseModel):
    name: str = Field(..., min_length=1, max_length=100) # using "..." is used by Pydantic to indicate that the field has no default value and is required
    price: float = Field(gt=0) # price must be greater than 0

product = Product(name="", price=5)

ValidationError: 1 validation error for Product
name
  ensure this value has at least 1 characters (type=value_error.any_str.min_length; limit_value=1)

## Using Unions

When using a plain Python `Union` you only get a static type hint with no automatic runtime checks. Pydantic, however, implements runtime validation and converstion logic based on those type hints. Pydantic will try to match the provided data against each type in the Union. If the data fits one of them (e.g. the fields match `PremiumFeatures`), it converts/validates accordingly.


In [ ]:
from typing import Union

class BaseUser(BaseModel):
    username: str
    email: str

class PremiumFeatures(BaseModel):
    max_storage_gb: int

class BasicFeatures(BaseModel):
    ads_enabled: bool

class Account(BaseModel):
    user: BaseUser
    tier: Union[PremiumFeatures, BasicFeatures]


# 1) Instantiating with the PremiumFeatures tier
premium_account = Account(
    user=BaseUser(username="alice", email="alice@example.com"),
    tier=PremiumFeatures(max_storage_gb=100)
)

print(premium_account)

# 2) Instantiating with the BasicFeatures tier
basic_account = Account(
    user=BaseUser(username="bob", email="bob@example.com"),
    tier=BasicFeatures(ads_enabled=True)
)

print(basic_account)

user=BaseUser(username='alice', email='alice@example.com') tier=PremiumFeatures(max_storage_gb=100)
user=BaseUser(username='bob', email='bob@example.com') tier=BasicFeatures(ads_enabled=True)
